In [133]:
import json
import os
from pprint import pprint
import re
import unicodedata
from unidecode import unidecode
import io
import shutil
import pandas as pd

In [123]:
data='../../../data/'
rdf_folder=data+'rdf0/'
json_folder=data+'json/'


In [124]:
def redif2jsonld(redif_file,path):
    ''' A function that gets a ReDIF input and converts it to a JSON-LD'''
    articles=[]
    article={}; 
    creators=[]
    creator={}
    with io.open(root+'/'+file,encoding='utf-8',errors='ignore') as f:
        content = f.readlines()

    # you may also want to remove whitespace characters like `\n` at the end of each line
    content = [x.strip() for x in content] 
    content = [x.replace('\x0c','?') for x in content]
    
    for i in content:
        pair=i.split(':')
        j=i.replace(pair[0]+':','').strip()
\            if len(creator)>0:
                creators.append(creator)
            creator={}
            creator['name']=j[:50]
        elif pair[0]=='Author-Email':
            creator['email']=j
        elif pair[0]=='Author-Workplace-Name':
            creator['affiliation']=j
        elif pair[0].lower()=='title':article['title']=j
        elif pair[0].lower()=='year':article['date']=j
        elif pair[0].lower()=='pages':article['pages']=j
        elif pair[0].lower()=='volume':article['volume']=j
        elif pair[0].lower()=='issue':article['issue']=j
        elif pair[0].lower()=='file-url':article['url']=j
        elif pair[0].lower()=='abstract':article['abstract']=j
        elif pair[0].lower()=='keywords':article['keywords']=[keyword.strip() for keyword in j.split(',')]
    jd=json.dumps(articles,ensure_ascii=False)
    #jd = unicode(jd, 'utf-8',errors='ignore')
    jl=json.loads(jd,encoding='utf-8')
    return (jl)

In [187]:
series_df=pd.read_csv('~/data/series.csv')
series_df['serie-handle']=series_df.handle.str[6:9]+series_df.handle.str[10:16]

In [192]:
shutil.rmtree(json_folder)
os.makedirs(json_folder)
i=0
for root, dirs, files in os.walk(rdf_folder):
    series=[]
    path = root.replace(rdf_folder,'').split(os.sep)
    if len(path)==2:
        i=i+1
        if (i % 1)==0: print i,
        jsonld_file=json_folder+root.replace('../../../data/rdf/','').replace('/','_')+'.json'
        serie={}
        serie['seriehandle']=path[0]+path[1]
        serie['providername']=series_df[series_df['serie-handle']==serie['seriehandle']].iloc[0]['provider-name']
        serie['seriename']=series_df[series_df['serie-handle']==serie['seriehandle']].iloc[0]['name']
        serie['items']=[]
        for file in files:        
            if file[-4:]=='.rdf' or file[-6:]=='.redif':
                json_data=redif2jsonld(file,path)   
            serie['items']=serie['items']+json_data
        series.append(serie)
        with open(jsonld_file, 'a') as outfile:
            json.dump(series, outfile)

1 2 3 4 5 6


In [193]:
with open(data+'mongo_json.json','a') as mongo_json:
    for json_file in os.listdir(data+'json'):
        if json_file[-5:]=='.json':
            with open(data+'json/'+json_file) as json_str:
                json_object=json.load(json_str)
                for item in json_object:
                    mongo_json.write(json.dumps(item)+'\n')

In [119]:
import pymongo
conn=pymongo.MongoClient('52.73.2.35:27017')
db=conn.repec

In [169]:
serie['serie_name']

'aahaarhec'

In [179]:
series_df[series_df['serie_name']==serie['serie_name']].iloc[0]

handle                                           RePEc:aay:wpaper
maintainer-email                                    info@aaye.org
name                    AAYE Policy Research Working Paper Series
provider-homepage                            http://www.aaye.org/
provider-institution                            RePEc:edi:aayeeea
provider-name             Association of African Young Economists
serie_name                                              aaywpaper
Name: 7, dtype: object

In [172]:
series_df[series_df['serie_name']=='aaywpaper']

TypeError: list indices must be integers, not str